In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
import math

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
##global variables
tz=0
tpr_by_fpr_min_thresh=0
target_column_name='Outcome'

In [4]:
data=pd.read_csv("datasets/diabetes.csv")

In [5]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
traindt=data;
test=data['Outcome']
traindt.drop('Outcome',axis=1,inplace=True)

In [7]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(traindt,test, test_size=0.25) # 70% training and 30% test

In [8]:
clf = svm.SVC(kernel="linear") # Linear Kernel

In [9]:

clf.fit(X_train, y_train)

SVC(kernel='linear')

In [10]:
y_pred = clf.predict(X_test)

In [11]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7708333333333334


In [12]:
import seaborn as sns
import matplotlib.pyplot as plt     

In [13]:
from sklearn.metrics import confusion_matrix

In [14]:
confusion_matrix(y_test,y_pred)

array([[104,  27],
       [ 17,  44]], dtype=int64)

In [15]:
# X_test

In [16]:
test_df_final=pd.concat([X_test,y_test],axis=1)


In [17]:
all_test=pd.concat([X_train,y_train],axis=1)

In [18]:
all_test['Pred']=clf.predict(X_train)

In [19]:
# all_test

In [20]:
true_predictions=all_test.loc[all_test['Outcome']==all_test['Pred']]
true_positive_predictions=true_predictions.loc[true_predictions['Pred']==1]
# true_positive_predictions
true_negative_predictons=true_predictions.loc[true_predictions['Pred']==0]
# true_negative_predictons

In [23]:
true_positive_predictions.drop('Pred',axis=1,inplace=True)
true_negative_predictons.drop('Pred',axis=1,inplace=True)

c:\Users\acer\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


<H1>3.Deleting features with 0 variance</H1>

In [24]:
true_predictions_x=true_predictions.drop(['Outcome','Pred'],axis=1)
true_predictions_y=true_predictions['Outcome']

In [25]:
true_predictions_x= true_predictions_x.loc[:, (true_predictions_x != true_predictions_x.iloc[0]).any()] 

In [26]:
list_of_features_with_non_0_var=list(true_predictions_x.columns)
list_of_features_with_non_0_var

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [27]:
true_negative_predictons.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [28]:
true_negative_predictons=true_negative_predictons[list_of_features_with_non_0_var+['Outcome']]
true_positive_predictions=true_positive_predictions[list_of_features_with_non_0_var+['Outcome']]


<h1>4.For each feature, compute the means for STP  and STN.</h1>

In [29]:

column_mean_for_STP_dict={}
column_mean_for_STN_dict={}
column_std_for_STP_dict={}
column_std_for_STN_dict={}
for column_name in list_of_features_with_non_0_var:
    column_mean_for_STP_dict[column_name]=true_positive_predictions[column_name].mean()
    column_mean_for_STN_dict[column_name]=true_negative_predictons[column_name].mean()
    column_std_for_STN_dict[column_name]=true_positive_predictions[column_name].std()
    column_std_for_STP_dict[column_name]=true_negative_predictons[column_name].std()





In [30]:
column_mean_for_STN_dict

{'Pregnancies': 3.115501519756839,
 'Glucose': 103.0273556231003,
 'BloodPressure': 67.19452887537994,
 'SkinThickness': 18.778115501519757,
 'Insulin': 52.6838905775076,
 'BMI': 28.82917933130699,
 'DiabetesPedigreeFunction': 0.40693009118541035,
 'Age': 30.4468085106383}

In [31]:
column_mean_for_STP_dict

{'Pregnancies': 5.222222222222222,
 'Glucose': 158.76923076923077,
 'BloodPressure': 72.11965811965813,
 'SkinThickness': 23.965811965811966,
 'Insulin': 128.77777777777777,
 'BMI': 37.26581196581196,
 'DiabetesPedigreeFunction': 0.5826581196581198,
 'Age': 39.47008547008547}

In [32]:
column_std_for_STN_dict

{'Pregnancies': 3.718803828613283,
 'Glucose': 25.408179722620357,
 'BloodPressure': 22.07829422179574,
 'SkinThickness': 19.20395596096141,
 'Insulin': 163.45030411629344,
 'BMI': 6.6152796275399615,
 'DiabetesPedigreeFunction': 0.4118342100809324,
 'Age': 11.77707639615655}

In [33]:
column_std_for_STP_dict

{'Pregnancies': 2.823902194556039,
 'Glucose': 21.75534373655603,
 'BloodPressure': 18.390376439466696,
 'SkinThickness': 14.372102298780742,
 'Insulin': 69.68524891628483,
 'BMI': 7.505294854223902,
 'DiabetesPedigreeFunction': 0.27114814257119435,
 'Age': 11.236281493249317}

<h1>6.Compute the Z for the difference between the two means computed in step 5, and define the set of most discriminative features F as those with ðZ > tz Þ.</h1>

In [34]:
num_of_samples_in_STP=len(true_positive_predictions.columns)
num_of_samples_in_STN=len(true_negative_predictons.columns)

In [35]:
def getZmeanDiff(columnName):
    num=column_mean_for_STN_dict[column_name]+column_mean_for_STP_dict[column_name]
    dnum=((column_std_for_STN_dict[column_name]**2)/num_of_samples_in_STN)+((column_mean_for_STP_dict[column_name]**2)/num_of_samples_in_STP)
    dnum=math.sqrt(dnum)
    return num/dnum


In [36]:
Z_diff_btw_two_means_dict={}
for column_name in list_of_features_with_non_0_var:
    Z_diff_btw_two_means_dict[column_name]=getZmeanDiff(column_name)

In [37]:
Z_diff_btw_two_means_dict

{'Pregnancies': 3.9015933041152624,
 'Glucose': 4.884585429428148,
 'BloodPressure': 5.541281720038652,
 'SkinThickness': 4.175464718109048,
 'Insulin': 2.6161558989082447,
 'BMI': 5.238923839947316,
 'DiabetesPedigreeFunction': 4.160783459562969,
 'Age': 5.092315365839051}

In [38]:
F=[]#
for column_name in list_of_features_with_non_0_var:
    if(Z_diff_btw_two_means_dict[column_name]>=tz):
        F+=[column_name]

In [39]:
F

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

<h1> 7.For each feature fi  F, find the threshold ðaibÞ that gives best TPR to FPR ratio over both STP and STN together. </h1>

In [40]:
def get_tpr_to_fpr_ratio(df,column_name,thresh,target_column_name):
    tp=0
    fp=0
    fn=0
    tn=0
    for index,row in df.iterrows():
        if(row[column_name]>=thresh):
            if(row[target_column_name]==1):
                tp+=1
            else:
                fp+=1
        else:
            if(row[target_column_name]==0):
                tn+=1
            else:
                fn+=1
    tpr=tp/(fn+tp)
    fpr=fp/(fn+tp)
    if fpr==0:
        fpr=10000000
    print("TPR:",tpr," Fpr:",fpr)
    return tpr/fpr

        
   
def get_aib_for_feature(column_name,df,step,target_column_name):
    myet=0
    aib=-1
    for poss_thres in range(int(math.floor(df[column_name].min())),int(math.ceil(df[column_name].max())),step):
        val=get_tpr_to_fpr_ratio(df,column_name,poss_thres,target_column_name)
        print("for thresh:",poss_thres," val:",val,"myet:", myet)
        if(myet<val):
            aib=poss_thres
            myet=val
    return aib





In [1]:
# tempf=['Pregnancies']
aib_dict_for_F={}
best_tpr_by_fpr_for_feature={}
for column_name in F:
    aib_dict_for_F[column_name]=get_aib_for_feature(column_name,true_predictions,1,target_column_name)
    best_tpr_by_fpr_for_feature[column_name]=get_tpr_to_fpr_ratio(true_predictions,column_name,aib_dict_for_F[column_name],target_column_name)

NameError: name 'F' is not defined

In [42]:
aib_dict_for_F

{'Pregnancies': 13,
 'Glucose': 158,
 'BloodPressure': 99,
 'SkinThickness': 55,
 'Insulin': 331,
 'BMI': 46,
 'DiabetesPedigreeFunction': 1,
 'Age': 49}

In [43]:
best_tpr_by_fpr_for_feature

{'Pregnancies': 4.0,
 'Glucose': 65.0,
 'BloodPressure': 1.3333333333333335,
 'SkinThickness': 2.9999999999999996,
 'Insulin': 10.0,
 'BMI': 11.999999999999998,
 'DiabetesPedigreeFunction': 1.6,
 'Age': 1.2}

<h1>8.Rank the features based on the values of the TPR to FPR ratio computed in step 7.</h1>

In [44]:
ranked_best_tpr_by_fpr_features=sorted(best_tpr_by_fpr_for_feature, key=lambda k: best_tpr_by_fpr_for_feature[k],reverse=True)

In [45]:
ranked_best_tpr_by_fpr_features

['Glucose',
 'BMI',
 'Insulin',
 'Pregnancies',
 'SkinThickness',
 'DiabetesPedigreeFunction',
 'BloodPressure',
 'Age']

<h1>9.For each feature fi from the ranked list in step 8, formulate a rule ri with fi and the corresponding threshold aib in the same order as that determined in step 8.</h1>

In [46]:
all_rule_list=[]
for column_name in ranked_best_tpr_by_fpr_features:
    temp_dict={
        "column_name":column_name,
        "sign":">",
        "thresh":aib_dict_for_F[column_name]
    }
    all_rule_list.append(temp_dict)
all_rule_list

[{'column_name': 'Glucose', 'sign': '>', 'thresh': 158},
 {'column_name': 'BMI', 'sign': '>', 'thresh': 46},
 {'column_name': 'Insulin', 'sign': '>', 'thresh': 331},
 {'column_name': 'Pregnancies', 'sign': '>', 'thresh': 13},
 {'column_name': 'SkinThickness', 'sign': '>', 'thresh': 55},
 {'column_name': 'DiabetesPedigreeFunction', 'sign': '>', 'thresh': 1},
 {'column_name': 'BloodPressure', 'sign': '>', 'thresh': 99},
 {'column_name': 'Age', 'sign': '>', 'thresh': 49}]

In [47]:
def isRuleSatisfied(rule_list,row):
    for rule in rule_list:
        cname=rule['column_name']
        if row[cname]<rule['thresh']:
            return False
    return True


 <h1>10.Starting with the first rule, compute the TPR from all STP that are correctly classified by that rule, as well as the FPR from all STN covered by this rule.</h1>

In [48]:
def get_tpr_by_fpr_for_rule_list(rule_list,df,target_column_name):
    tp=0
    tn=0
    fp=0
    fn=0
    for index,row in df.iterrows():
        
        if(isRuleSatisfied(rule_list,row)):
            if(row[target_column_name]==1):
                tp+=1
            else:
                fp+=1
        else:
            if(row[target_column_name]==0):
                tn+=1
            else:
                fn+=1
    tpr=tp/(fn+tp)
    fpr=fp/(fn+tp)
    print("tp:",tp," fp:",fp," tn:",tn," fn:",fn,"TPR:",tpr," Fpr:",fpr)
    if fpr==0:
        return (tpr/2000000,0)
   
    return (tpr/fpr,fpr)
    
    
    

In [49]:
final_rule_list=[]

In [50]:
# def add_rule_rec(df,all_rule_list,pres_ind,pres_rule_list,last_tpr_by_fpr_ratio,target_column_name):
def delete_rows_satisfying_rule(df,rule_list,target_column_name):
    list_of_ind_to_drop=[]
    ind=0
    for index,row in df.iterrows():
        # print("ind:",index)
    
        if(isRuleSatisfied(rule_list,row)):
            if(row[target_column_name]==1):
                list_of_ind_to_drop+=[ind]
        else:
            if(row[target_column_name]==0):
                list_of_ind_to_drop+=[ind]
        ind+=1
    df.drop(df.index[[list_of_ind_to_drop]],inplace=True)
    return df
    



In [51]:
result_rule_list=[]
df=true_predictions.copy()
for init_rule_ind in range(0,len(all_rule_list)):
    init_rule_list=[all_rule_list[init_rule_ind]]
    (init_tpr_by_fpr,_)=get_tpr_by_fpr_for_rule_list(init_rule_list,df,'Outcome')
    print("rule list:", init_rule_list)
    print("tpr/fppr:",init_tpr_by_fpr)
    print("df shape::",df.shape)
    new_rule_list=init_rule_list.copy()
    last_ratio=tpr_by_fpr_min_thresh
    vis=[0]*len(all_rule_list)
    vis[init_rule_ind]=1
    max_tpr_fpr_ratio=init_tpr_by_fpr
    max_tpr_fpr_ratio_ind=-1
    while get_tpr_by_fpr_for_rule_list(new_rule_list,df,'Outcome')[0]> last_ratio:
        init_rule_list=new_rule_list
        max_tpr_fpr_ratio=0
        max_tpr_fpr_ratio_ind=-1
        for new_rule_ind in range(len(all_rule_list)):
            if(vis[new_rule_ind]==1):
                continue
            temp_new_rule_list=new_rule_list.copy()
            temp_new_rule_list+=[all_rule_list[new_rule_ind]]
            (new_tpr_by_fpr,fpr_for_list)=get_tpr_by_fpr_for_rule_list(temp_new_rule_list,df,'Outcome')
            print("for ind:",new_rule_ind, "new rat:",new_tpr_by_fpr," fpr_for_list:",fpr_for_list)
            if new_tpr_by_fpr>max_tpr_fpr_ratio and fpr_for_list!=0:
                max_tpr_fpr_ratio=new_tpr_by_fpr
                max_tpr_fpr_ratio_ind=new_rule_ind
        print("new tpr/fpr",max_tpr_fpr_ratio)
        if max_tpr_fpr_ratio_ind==-1:
            break
        # if max_tpr_fpr_ratio>tpr_by_fpr_min_thresh:
        
        vis[max_tpr_fpr_ratio_ind]=1
        new_rule_list+=[all_rule_list[max_tpr_fpr_ratio_ind]]
        
    print( "init rule list:",init_rule_list)
    result_rule_list.append(init_rule_list)
    #removing rows from df which are already classified
    # if max_tpr_fpr_ratio_ind==-1:
    #     continue
    df=delete_rows_satisfying_rule(df,new_rule_list,"Outcome")
    print("df shape:",df.shape)
            
        



    



tp: 65  fp: 1  tn: 328  fn: 52 TPR: 0.5555555555555556  Fpr: 0.008547008547008548
rule list: [{'column_name': 'Glucose', 'sign': '>', 'thresh': 158}]
tpr/fppr: 65.0
df shape:: (446, 10)
tp: 65  fp: 1  tn: 328  fn: 52 TPR: 0.5555555555555556  Fpr: 0.008547008547008548
tp: 3  fp: 0  tn: 329  fn: 114 TPR: 0.02564102564102564  Fpr: 0.0
for ind: 1 new rat: 1.282051282051282e-08  fpr_for_list: 0
tp: 7  fp: 0  tn: 329  fn: 110 TPR: 0.05982905982905983  Fpr: 0.0
for ind: 2 new rat: 2.991452991452992e-08  fpr_for_list: 0
tp: 0  fp: 0  tn: 329  fn: 117 TPR: 0.0  Fpr: 0.0
for ind: 3 new rat: 0.0  fpr_for_list: 0
tp: 3  fp: 0  tn: 329  fn: 114 TPR: 0.02564102564102564  Fpr: 0.0
for ind: 4 new rat: 1.282051282051282e-08  fpr_for_list: 0
tp: 9  fp: 0  tn: 329  fn: 108 TPR: 0.07692307692307693  Fpr: 0.0
for ind: 5 new rat: 3.846153846153846e-08  fpr_for_list: 0
tp: 3  fp: 0  tn: 329  fn: 114 TPR: 0.02564102564102564  Fpr: 0.0
for ind: 6 new rat: 1.282051282051282e-08  fpr_for_list: 0
tp: 17  fp: 1  t

c:\Users\acer\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexes\base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


df shape: (88, 10)
tp: 7  fp: 0  tn: 0  fn: 81 TPR: 0.07954545454545454  Fpr: 0.0
rule list: [{'column_name': 'Insulin', 'sign': '>', 'thresh': 331}]
tpr/fppr: 3.9772727272727275e-08
df shape:: (88, 10)
tp: 7  fp: 0  tn: 0  fn: 81 TPR: 0.07954545454545454  Fpr: 0.0
tp: 5  fp: 0  tn: 0  fn: 83 TPR: 0.056818181818181816  Fpr: 0.0
for ind: 0 new rat: 2.8409090909090908e-08  fpr_for_list: 0
tp: 0  fp: 0  tn: 0  fn: 88 TPR: 0.0  Fpr: 0.0
for ind: 1 new rat: 0.0  fpr_for_list: 0
tp: 0  fp: 0  tn: 0  fn: 88 TPR: 0.0  Fpr: 0.0
for ind: 3 new rat: 0.0  fpr_for_list: 0
tp: 0  fp: 0  tn: 0  fn: 88 TPR: 0.0  Fpr: 0.0
for ind: 4 new rat: 0.0  fpr_for_list: 0
tp: 2  fp: 0  tn: 0  fn: 86 TPR: 0.022727272727272728  Fpr: 0.0
for ind: 5 new rat: 1.1363636363636364e-08  fpr_for_list: 0
tp: 0  fp: 0  tn: 0  fn: 88 TPR: 0.0  Fpr: 0.0
for ind: 6 new rat: 0.0  fpr_for_list: 0
tp: 0  fp: 0  tn: 0  fn: 88 TPR: 0.0  Fpr: 0.0
for ind: 7 new rat: 0.0  fpr_for_list: 0
new tpr/fpr 0
init rule list: [{'column_name':

In [52]:
result_rule_list

[[{'column_name': 'Glucose', 'sign': '>', 'thresh': 158},
  {'column_name': 'Age', 'sign': '>', 'thresh': 49}],
 [{'column_name': 'BMI', 'sign': '>', 'thresh': 46}],
 [{'column_name': 'Insulin', 'sign': '>', 'thresh': 331}],
 [{'column_name': 'Pregnancies', 'sign': '>', 'thresh': 13}],
 [{'column_name': 'SkinThickness', 'sign': '>', 'thresh': 55}],
 [{'column_name': 'DiabetesPedigreeFunction', 'sign': '>', 'thresh': 1}],
 [{'column_name': 'BloodPressure', 'sign': '>', 'thresh': 99}],
 [{'column_name': 'Age', 'sign': '>', 'thresh': 49}]]

<h1> Post Pruning</h1>

In [53]:
def get_rule_list_predictions(rule_list,df):
    y_pred=[]
    for index,row in df.iterrows():
        for rule in rule_list:
            if isRuleSatisfied(rule,row):
                y_pred+=[1]
                break
        else:
            y_pred+=[0]
           
    return y_pred

                
    

In [54]:
from sklearn import metrics

In [80]:
post_pruned_rule_list=[]
last_auc_area=0
for rule in result_rule_list:
    temp_y_pred=get_rule_list_predictions(post_pruned_rule_list+[rule],true_predictions)
    # print(temp_y_pred)
    # print(true_predictions)
    # print(" pred len:",len(temp_y_pred))
    fpr, tpr, thresholds = metrics.roc_curve(true_predictions['Outcome'],temp_y_pred)
    pres_auc=metrics.auc(fpr,tpr)
    print("pres auc",pres_auc)
    if(pres_auc-last_auc_area>=0.04):
        post_pruned_rule_list.append(rule)
        last_auc_area=pres_auc

pres auc 0.5711298158106668
pres auc 0.6208921102538124
pres auc 0.6492868833294365
pres auc 0.6364663705089236
pres auc 0.6208921102538124
pres auc 0.6612500974203102
pres auc 0.6609643311770971
pres auc 0.6675109760216144


In [81]:
post_pruned_rule_list

[[{'column_name': 'Glucose', 'sign': '>', 'thresh': 158},
  {'column_name': 'Age', 'sign': '>', 'thresh': 49}],
 [{'column_name': 'BMI', 'sign': '>', 'thresh': 46}],
 [{'column_name': 'DiabetesPedigreeFunction', 'sign': '>', 'thresh': 1}]]

In [82]:
def check_accuracy_of_rules_for_dataframe(rule_list,df,target_column_name):
    fp=0
    fn=0
    tp=0
    tn=0
    for index,row in df.iterrows():
        for rule in rule_list:
            if isRuleSatisfied(rule,row):
                if row[target_column_name]==1:
                    tp+=1
                else:
                    fp+=1
                break
        else:
            if row[target_column_name]==0:
                tn+=1
            else:
                fn+=1
    return (tp,tn,fp,fn)

In [83]:
(tp,tn,fp,fn)=check_accuracy_of_rules_for_dataframe(post_pruned_rule_list,test_df_final,'Outcome')

In [84]:
print("Accuracy of the rules:",(tp+tn)/(tp+tn+fp+fn))

Accuracy of the rules: 0.6666666666666666


In [85]:
len(clf.support_vectors_)

301

In [86]:
def print_rule_list(rule_list):
    final_res="\n"
    for rules in rule_list:
        res="IF "
        # print("r:",rules[0])
        for i in range(0,len(rules)):
            if i>0:
                res+=" AND "
            res+=rules[i]['column_name']
            res+=" "
            res+=rules[i]['sign']
            res+=" "
            res+=str(rules[i]['thresh'])
        res+=" THEN OUTCOME IS 1"
        # print("res:",res)
        final_res+="\n"
        final_res+=res
    print(final_res)        

In [87]:
print_rule_list(post_pruned_rule_list)



IF Glucose > 158 AND Age > 49 THEN OUTCOME IS 1
IF BMI > 46 THEN OUTCOME IS 1
IF DiabetesPedigreeFunction > 1 THEN OUTCOME IS 1
